### starting a basic sequence to sequence LSTM model 

In [4]:
import tensorflow as tf
import numpy as np

In [9]:
class TimeSeriesEncoder(tf.keras.Model):
  def __init__(self, units):
    super().__init__()
    self.lstm = tf.keras.layers.LSTM(units, return_sequences=True, return_state=True)

  def call(self, inputs, state=None):
    outputs, state_h, state_c = self.lstm(inputs, initial_state=state)
    return outputs, (state_h, state_c)

class TimeSeriesDecoder(tf.keras.Model):
  def __init__(self, units):
    super().__init__()
    self.lstm = tf.keras.layers.LSTM(units, return_sequences=True)
    self.dense = tf.keras.layers.Dense(1)  # One output for predicted future value

  def call(self, hidden_state):
    outputs = self.lstm(None, initial_state=hidden_state)  # Pass None for inputs
    predictions = self.dense(outputs)
    return predictions

class Seq2SeqLSTMForcast(tf.keras.Model):
  def __init__(self, encoder_units, decoder_units):
    super().__init__()
    self.encoder = TimeSeriesEncoder(encoder_units)
    self.decoder = TimeSeriesDecoder(decoder_units)

  def call(self, inputs):
    encoder_outputs, encoder_hidden = self.encoder(inputs)

    predictions = []
    for _ in range(self.target_steps):  # Predict for target_steps (use internal attribute)
      decoder_output = self.decoder(tf.expand_dims(encoder_outputs[:, -1, :], 0), encoder_hidden)
      predictions.append(decoder_output)

    predictions = tf.concat(predictions, axis=1)  # Concatenate predictions
    return predictions
  
  # Generate random time series data function (replace with your data source)
def generate_random_data(samples, timesteps, features):
  data = np.random.rand(samples, timesteps, features)
  # Optionally add trends, seasonality, or external factors here

  return data.astype(np.float32)

# Example usage
model = Seq2SeqLSTMForcast(encoder_units=256, decoder_units=128)

# Define loss function and optimizer
loss_function = tf.keras.losses.MeanSquaredError()  # MSE for forecasting
optimizer = tf.keras.optimizers.Adam()

# Generate sample data (replace with your actual data)
past_data = generate_random_data(samples=100, timesteps=20, features=7)
future_values = past_data[:, -1:, :]  # Take last timestep as future value


In [19]:
model.compile(loss=loss_function,optimizer=optimizer)

model.fit(model.fit([past_data, past_data], past_data))



AttributeError: Exception encountered when calling LSTM.call().

[1m'LSTMCell' object has no attribute 'kernel'[0m

Arguments received by LSTM.call():
  • sequences=tf.Tensor(shape=(None, 20, 7), dtype=float32)
  • initial_state=None
  • mask=None
  • training=False

In [16]:
future_values.shape

(100, 1, 7)

In [15]:
with tf.GradientTape() as tape:
  predictions = model(past_data, target_steps=1)
  loss = loss_function(future_values, predictions)

gradients = tape.gradient(loss, model.trainable_variables)
optimizer.apply_gradients(zip(gradients, model.trainable_variables))

TypeError: got an unexpected keyword argument 'target_steps'